In [1]:
%cd ../src

/Users/mikhail_zybin/food-order-chat-bot/src


In [2]:
%load_ext autoreload
%autoreload 2

import evaluator
from pathlib import Path
import seaborn as sns
import numpy as np

In [ ]:
factual_correctness_list = []
appropriateness_list = []
for i in range(1, 20):
    factual_correctness, appropriateness = evaluator.evaluate_scenario(i)
    factual_correctness_list.append(factual_correctness)
    appropriateness_list.append(appropriateness)
    print(factual_correctness_list, np.mean(np.array(factual_correctness_list)), np.std(np.array(factual_correctness_list)))
    print(appropriateness_list, np.mean(np.array(appropriateness_list)), np.std(np.array(appropriateness_list)))
    print("#"*20)

2024-12-23 20:26:24.291 | INFO     | evaluator:evaluate_scenario:60 - Evaluating scenario 1
2024-12-23 20:26:29.334 | DEBUG    | chatbot:parse_llm_json:29 - {
  "dish_names": ["Pasta"],
  "dish_quantities": [1],
  "restaurant_name": "",
  "delivery_time": "",
  "explanation": "The user has chosen a specific dish, but has not chosen a restaurant yet."
}
2024-12-23 20:26:35.991 | INFO     | evaluator:evaluate_scenario:79 - --------------------predicted_message--------------------
2024-12-23 20:26:35.994 | INFO     | evaluator:evaluate_scenario:80 - Pasta sounds delicious! Unfortunately, I couldn't find any pasta dishes in the available restaurants. However, I can suggest some restaurants that might have similar dishes.

Based on your interest in pasta, I recommend checking out the following restaurants:

1. **Roman Holiday** - They have a variety of Italian dishes, including Spaghetti Carbonara (pasta) for $14.99.
2. **Sukhothai** - They have a dish called Chicken Pad See Ew, which is a 

In [ ]:
sns.barplot(factual_correctness_list)

In [ ]:
sns.barplot(appropriateness_list)


In [ ]:
evaluator.evaluate_scenario(6)

In [ ]:
evaluator.evaluate_scenario(9)